### Introduction 

In the following code the customer review data is collected from the html through the process of the web scrapping. The three selected categories are Atomotive, Fashion and Gym. The customer review is collected with the Star rating . This star rating is used to decide the positive and negative nature of the review.The star rating 4 and above is consider as positive and below that is consider as negative. The textual review is also collected for data minig purpose and the performance of text classification algorithm is evaluted in the later part of the assignment.

In [22]:
#Importing the required librabries in the assignament.
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from lxml import html
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#Decalaration of the variables
starRating=[]
natureOfReview=[]
reviewComment=[]
Fashion_review_data = []
Automotive_review_data = []
Gym_review_data = []
#Create a Gaussian Classifier

## Data Collection

In this part we are collecting the HTML data from the given URL


In [23]:
# This function will collect all the data of star rating and review comments from the URL 
def get_rating_status (URLReview):
    
    URLpage = requests.get(URLReview) #The URL of the comapny review is accessed through request 
    reviewSoup = BeautifulSoup(URLpage.content, 'html.parser')#BeautifulreviewSoup is used to parse the data
    trimhtml = reviewSoup.find(id='all')# It will trim data under the tag id= all in html
    reviewtrim=trimhtml.find_all(class_='review')# All the class with review name is accessed
    
    for t in reviewtrim:           # Each elecment of the class review name is accessed
        ratingStar = t.find('img')    # this will search for the tag with img
        if ratingStar:               # It will check for the img tag
            temp = ratingStar['alt'] # It will search for the alt tag
            starRating.append(temp)  #This is to store the star rating
            if(temp=='5-star' or temp=='4-star' ): #This will check if the review is positive or negative
                natureOfReview.append(1)         #For positive review 1 will be stored in the list
            else:
                natureOfReview.append(0)        #For negative review 0 will be stored in the list
        #The review comment is stored in the another list with lower case (Preprocessing)
        reviewComment.append(t.find(class_='review-text').get_text().lower())
    

In [24]:
#In this function the catagory URL will be input and then it wil access the review of the comapnies in particular catagory
def Catagory_data (URLcatagory):
    URLcatagorypage = requests.get(URLcatagory) #The URL of the category accessed through request  
    catagorysoup = BeautifulSoup(URLcatagorypage.content, 'html.parser')#BeautifulreviewSoup is used to parse the data
    trimhtmllink = catagorysoup.find(id='all')# It will trim data under the tag id= all in html
    reviewtrimlink =trimhtmllink.find_all('p')# All the p tag data is accessed
    for t in reviewtrimlink:         #For each data with p tag is accessed
        ratingStar = t.find('a')     # searching for the tag a
        if ratingStar:               #this is the condition for a tag
            temp = ratingStar['href']#Accessing the data under href tag
            URL = 'http://mlg.ucd.ie/modules/yalp/'+ temp #The URL of companies review is generated  
            get_rating_status(URL)  #Call the function with the URL of companies review to collect the data

In [25]:
#This functio will generate the CSV file of the data collected from the html with category and data to be stored as an input
def Generating_CSV (category,zip_data_to_write):
    
    with open('ReviewData'+category+'.csv', 'w', encoding='utf-8') as csvfile: #creating the csv file with customized name
        fileInWritemode = csv.writer(csvfile) #openeing CSV file in write mode
        count=0        # counter decalraition
        for word in zip_data_to_write: #accessing the data to write in csv
            if count != 0:            #This will start from the second row
                fileInWritemode.writerow(word) #Writing row
            else:                     #generating column with heading
                fileInWritemode.writerow(['Star Rating','Nature of Rating','Review Comment'])#Names of the column
                count = count+1     #Update the counter value
    csvfile.close()   #save and closing the file
   

In [26]:
# collection of data and generating csv file for Automotive category
Automotive_URL='http://mlg.ucd.ie/modules/yalp/automotive_list.html'
Catagory_data (Automotive_URL) #calling function with category URL 

Automotive_review_data.append(starRating) #Appending the data in the list from the function with star rating
Automotive_review_data.append(natureOfReview) #Appending the data in the list from the function with Nature of Review
Automotive_review_data.append(reviewComment) #Appending the data in the list from the function with Review comments
zip_Automotive_data_to_write = zip(*Automotive_review_data) # converting the list into zip file

# clearing the list decalred globally
starRating=[]
natureOfReview=[]
reviewComment=[]

Generating_CSV ('Automotive',zip_Automotive_data_to_write)# calling the function to genrate the csv file with category and data to be written as argument

In [27]:
# collection of data and generating csv file for Fashion category
Fashion_URL='http://mlg.ucd.ie/modules/yalp/fashion_list.html'
Catagory_data (Fashion_URL)  #calling function with category URL

Fashion_review_data.append(starRating) #Appending the data in the list from the function with star rating
Fashion_review_data.append(natureOfReview) #Appending the data in the list from the function with Nature of Review
Fashion_review_data.append(reviewComment) #Appending the data in the list from the function with Review comments
zip_Fashion_data_to_write = zip(*Fashion_review_data)       # converting the list into zip file

# clearing the list decalred globally
starRating=[]
natureOfReview=[]
reviewComment=[]

Generating_CSV ('Fashion',zip_Fashion_data_to_write)# calling the function to genrate the csv file with category and data to be written as argument

In [28]:
# collection of data and generating csv file for Gym category
Gym_URL='http://mlg.ucd.ie/modules/yalp/gym_list.html'
Catagory_data (Gym_URL) #calling function with category URL

Gym_review_data.append(starRating) #Appending the data in the list from the function with star rating
Gym_review_data.append(natureOfReview) #Appending the data in the list from the function with Nature of Review
Gym_review_data.append(reviewComment) #Appending the data in the list from the function with Review comments
zip_Gym_data_to_write = zip(*Gym_review_data)         # converting the list into zip file

# clearing the list decalred globally
starRating=[]
natureOfReview=[]
reviewComment=[]

Generating_CSV ('Gym',zip_Gym_data_to_write)# calling the function to genrate the csv file with category and data to be written as argument

## Text Preprocessing

In this section the text data collected from the review comment is preprocessed.On part of preprocessing is to convert the data in lower case is already done when storing the review comment in the csv file.
The text preprocessing is an important step in improving the performance of the bag of words model. The steps which are included in text preprocessing are as given below.


1.Case conversion:- The incoming data is stored in lower case in csv file


2.Filter short term:- In this case the words which are less than 4 are filtered out


3.Filter Stop words:- In the vectorizer the all the english stop words are filtered using english                           command


4.Filter infrequent word


5.Along with this in order give more weight to more important terms term frequency–inverse document frequency (TF-IDF).

In [8]:
def Data_preprocessing(FileName,catagory):
    CombineCsv_data = pd.read_csv(FileName) #Reading the combine csv data file'
    Reviewtext = CombineCsv_data["Review Comment"] #fetching the data from the column Review Comment
    natureOfComment = CombineCsv_data["Nature of Rating"] #fetching the data from the column Nature of Rating
    
    # Tokenization of the data with preprocessing steps
    vectorizer = TfidfVectorizer(stop_words="english",min_df = 4)
    vectorizedData = vectorizer.fit_transform(Reviewtext)
    
    #Dividing the data into traning and testing dataset 
    trainingData, testingData, natureOfComment_train, natureOfComment_test = train_test_split(vectorizedData, natureOfComment, test_size=0.3)
    
    print("Number of samples in traning data  is %d"%trainingData.shape[0]+" for "+ catagory )
    print("Number of testing samples is %d "%testingData.shape[0]+" for "+ catagory )
    
    #Classification model of Logistic Regression
    logisticRegr = LogisticRegression()
    logisticRegr.fit(trainingData,natureOfComment_train)#Training the model with train data
    predicted = logisticRegr.predict(testingData)       # Predicting the test data
    print("Accuracy for "+catagory+ " review from logistic regression = %.2f" % accuracy_score(natureOfComment_test, predicted) ) #Accuracy of the model
    print("Precision for "+catagory+ " review from logistic regression = %.2f" % precision_score(natureOfComment_test, predicted, pos_label=1) )#Precision of the Model
    print("Recall for "+catagory+ " review from logistic regression = %.2f" % recall_score(natureOfComment_test, predicted, pos_label=1) )# Recall of the model

## Text Classification

In the text classification the text is used to identify the the nature of the speech.In this case we will try to identify whether the review is positive or negative. In order to achieve this we have to train a model with some training data . Then the testing data is used for predection.There are different classification algorithms which can be used to classify the algorithm.


For this assingment the Logistic Regression algorithm is used. Logstic Regression uses sigmoid function for the predection.The sigmoid function maps any real value to value between 0 to 1. This strategy makes the model more accurate for prediction.

In [9]:

Data_preprocessing('ReviewDataAutomotive.csv','Automotive')


#data_train_auto, data_test_auto, target_train_auto, target_test_auto = train_test_split(Xdata, target, test_size=0.3)

Number of samples in traning data  is 1399 for Automotive
Number of testing samples is 600  for Automotive
Accuracy for Automotive review from logistic regression = 0.92
Precision for Automotive review from logistic regression = 0.91
Recall for Automotive review from logistic regression = 0.95


### Evaluation Strategy

There are different evaluation strategy which can be used to evaluate the performance of the buld model.For this assignment the Accrracy is choosed for performance of the algorithm.

In [10]:
Data_preprocessing('ReviewDataFashion.csv','Fashion')

Number of samples in traning data  is 1399 for Fashion
Number of testing samples is 600  for Fashion
Accuracy for Fashion review from logistic regression = 0.88
Precision for Fashion review from logistic regression = 0.85
Recall for Fashion review from logistic regression = 0.97


This accuracy shows how accurate the predection of the model.

In [11]:
Data_preprocessing('ReviewDataGym.csv','Gym')

Number of samples in traning data  is 1399 for Gym
Number of testing samples is 600  for Gym
Accuracy for Gym review from logistic regression = 0.85
Precision for Gym review from logistic regression = 0.82
Recall for Gym review from logistic regression = 0.96


Above value shows that the algorithm is well trained as it gives high value of accuracy. The predictions made by the algorithm is highly accurate

Now the following is calculation of the performance of the model with training data of one catagory and testing data of another catagory

#### Collecting data from Automotive Catagory

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
csv_data_Auto = pd.read_csv('ReviewDataAutomotive.csv') #Reading the combine csv data file'
ReviewData_Auto = csv_data_Auto["Review Comment"] #fetching the data from the column Review Comment
ratingNature_Auto = csv_data_Auto["Nature of Rating"] #fetching the data from the column Nature of Rating

# Tokenization of the data with preprocessing steps
vectorizer_Auto = TfidfVectorizer(stop_words="english",min_df = 4)
preprocessedData_Auto = vectorizer_Auto.fit_transform(ReviewData_Auto)

data_train_Auto, data_test_Auto, target_train_Auto, target_test_Auto = train_test_split(preprocessedData_Auto, ratingNature_Auto)

#### Collecting data from Fashion Catagory

In [13]:
csv_data_Fash = pd.read_csv('ReviewDataFashion.csv')#Reading the combine csv data file'
ReviewData_Fash = csv_data_Fash["Review Comment"] #fetching the data from the column Review Comment
ratingNature_Fash = csv_data_Fash["Nature of Rating"] #fetching the data from the column Nature of Rating

# Tokenization of the data with preprocessing steps
preprocessedData_Fash = vectorizer_Auto.transform(ReviewData_Fash)

data_train_Fash, data_test_Fash, target_train_Fash, target_test_Fash = train_test_split(preprocessedData_Fash, ratingNature_Fash, test_size=0.3)

#### Collecting data from Gym Catagory

In [14]:
csv_data_Gym = pd.read_csv('ReviewDataGym.csv') #Reading the combine csv data file'
ReviewData_Gym = csv_data_Gym["Review Comment"] #fetching the data from the column Review Comment
ratingNature_Gym = csv_data_Gym["Nature of Rating"] #fetching the data from the column Nature of Rating

# Tokenization of the data with preprocessing steps
preprocessedData_Gym = vectorizer_Auto.transform(ReviewData_Gym)

data_train_Gym, data_test_Gym, target_train_Gym, target_test_Gym = train_test_split(preprocessedData_Gym, ratingNature_Gym)

####  1. Performance of Logistic Regression model with training data of Automotive and testing data of Fashion and Gym catagory

In [15]:
AutoLis=[]
logisticRegr = LogisticRegression()

logisticRegr.fit(data_train_Auto,target_train_Auto) #Training the model with train data with Automotive category
predicted_c1 = logisticRegr.predict(data_test_Fash) # Predicting the test data of Fashion
AutoLis.append("Automotive Train Data")
AutoLis.append("")
AutoLis.append(accuracy_score(target_test_Fash, predicted_c1))
print("Accuracy of Automotive with Fashion = %.2f" % accuracy_score(target_test_Fash, predicted_c1) )

Accuracy of Automotive with Fashion = 0.87


In [16]:
predicted_c2 = logisticRegr.predict(data_test_Gym) # Predicting the test data of Gym
AutoLis.append(accuracy_score(target_test_Gym, predicted_c2))
print("Accuracy of Automotive with Gym = %.2f" % accuracy_score(target_test_Gym, predicted_c2) )

Accuracy of Automotive with Gym = 0.85


#### 2. Performance of Logistic Regression model with training data of Fashion and testing data of Automotive and Gym catagory

In [17]:
fashList=[]
logisticRegr.fit(data_train_Fash,target_train_Fash) #Training the model with train data with Fashion category
predicted_c1 = logisticRegr.predict(data_test_Auto) # Predicting the test data of Automotive
fashList.append("Fashion Train Data")
fashList.append(accuracy_score(target_test_Auto, predicted_c1))
fashList.append("")
print("Accuracy of Fashion with Automotive = %.2f" % accuracy_score(target_test_Auto, predicted_c1) )

Accuracy of Fashion with Automotive = 0.86


In [18]:
predicted_c1 = logisticRegr.predict(data_test_Gym) # Predicting the test data of Gym
fashList.append(accuracy_score(target_test_Gym, predicted_c1))
print("Accuracy of Fashion with Gym= %.2f" % accuracy_score(target_test_Gym, predicted_c1) )

Accuracy of Fashion with Gym= 0.89


#### 3. Performance of Logistic Regression model with training data of Gym and testing data of Automotive and Fashion catagory

In [19]:
Gymlist=[]
logisticRegr.fit(data_train_Gym,target_train_Gym) #Training the model with train data with Gym category
predicted_c1 = logisticRegr.predict(data_test_Auto) # Predicting the test data of Automotive
Gymlist.append("Gym Train Data")
Gymlist.append(accuracy_score(target_test_Auto, predicted_c1))
print("Accuracy of Gym with Automotive = %.2f" % accuracy_score(target_test_Auto, predicted_c1) )

Accuracy of Gym with Automotive = 0.86


In [20]:
predicted_c1 = logisticRegr.predict(data_test_Fash) # Predicting the test data of Fashion
Gymlist.append(accuracy_score(target_test_Fash, predicted_c1))
Gymlist.append("")

print("Accuracy of Gym with Fashion= %.2f" % accuracy_score(target_test_Fash, predicted_c1) )

Accuracy of Gym with Fashion= 0.86


In [21]:
from astropy.table import QTable, Table, Column
colname=[]
colname.append("")
colname.append("Automotive Test Data")
colname.append("Fashion Test Data")
colname.append("Gym Test Data")
arr = (colname,AutoLis ,fashList ,Gymlist)
Table(arr)

col0,col1,col2,col3
str20,str21,str18,str18
,Automotive Train Data,Fashion Train Data,Gym Train Data
Automotive Test Data,,0.862,0.858
Fashion Test Data,0.8733333333333333,,0.8616666666666667
Gym Test Data,0.854,0.892,


Above is the table of showing Accuracy columns with the category train data and rows with the category test data.

The value of the accuracy obtained shows that even though data is trained in any catogory it can predict the data from another catagory with high accuracy. This tells that the build model is reliabel.

## Conclusion


This assingment gave an insight about the different steps in any Data science project. Starting with the data extraction form the html page. During extraction process it is important to figure out the data which carrys more information. After extracting the data it is important to store it in proper format so that it can be extracted and utlize for further processing. The obtained data has need to convert into the format so that it can be evaluated(Tokenization). There are different pre processing techniques which helps to extract the necessary data. This data can be classified using some algorithms.The performance of the algorithm can be evaluated using traning data and testing data of different catagories.